In [3]:
from LFOFM import *
import threading
import OSC

wp = WavePlayer()
tempo = wp.setTempo

import PydalChanel as pydalModule

read = pydalModule.read
pydal = pydalModule.getPydalInstance()
port = pydal.port
ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)

triggerFunctions = {}

def sendChord(chord):
    scClient = pydal.superColliderClient
    msg = OSC.OSCMessage()
    msg.setAddress("/waveNotes")
    msg.append(chord)
    scClient.send(msg)
    

def funcTriggerResponder(addr, tags, stuff, source):
    if stuff[0] in triggerFunctions:
        triggerFunctions[stuff[0]]()
        #could have the "drumStr" be something like "a_param1_param2_..._" and feed string to function,
        #splitting off the first "_" for the function id

pydal.superColliderServer.addMsgHandler("/funcTrigger", funcTriggerResponder)

#### bugs discovered 
- subtracting waves from each other (subtraction generally?)
- (fixed) variables don't work normally - operations modify the objects so arrithmetic doesn't return new stuff

#### ideas
- rather than quantized midi notes, wave sweeps through notes of an arpeggiated chord. 
    - implementation: if the rounded midi note is not in the chord, don't send anything
- implement a small tree of chords, random walk them
- implement a small tree of pitch and q waves, random walk them 
- time before a random walk step will follow some particular randomly generated sequence determined at the start of the piece
- (after other two are working) coin-flip to whether the pitch and q waves will interpolate between walk steps (1/2 of step time is taken to transition to next waves, 1/2 is spent waiting at them). 

#### implementation
- add handler to LFOFM.scd to send chord values to (will be universal across waves for now) 
- using pydal funcTrigger type (but with pattern constructed with LoopPattern), trigger random walks 
- allow system to do interpolation
    - create a new channel type like funcTrigger that, upon a "hit", sets off a task to stream interpolation values to python

In [4]:
from LFOFM import *
from treeBuilder import TreeBuilder
import random

class Counter:
    def __init__(self):
        self.count = 0
    def __call__(self):
        count = self.count
        self.count += 1
        return self.count 

freqs = [Sin(2./(i+1))*4 + 35+(i*5) for i in range(8)]
qs = [Sin(0.7/(i+1))*(i+1)*3 + i*7 + 3 for i in range(8)]
#simply have 3 different walkers (one for each voice's freq/q), and force coordinate them every once in a while?

waveTrees = [0, 0, 0]
for i in range(len(waveTrees)):
    c = Counter()
    getWaves = lambda ind: [freqs[ind], qs[ind]]
    waveTrees[i] = TreeBuilder([freqs[0], qs[0]], lambda a: getWaves(c()), fullTreeDepth=2)

c = Counter()
chords = [random.sample(range(12), 3) for i in range(8)]
chordTree = TreeBuilder(chords[0], lambda a: chords[c()], fullTreeDepth=2)
    

def randWalk():
    return random.choice(["_"]*3 + ["\/:0", "\/:1", ">", "<", "^", "^"])


def wavesWalk():
    for i in range(len(waveTrees)):
        waveTree = waveTrees[i]
        waveTree.execute(randWalk())
        waves = waveTree.currentNode.value
        wp.startWaveOSC('f'+str(i+1), '/'+str(i+1)+'/freq', waves[0], metaInfo="warp")
        wp.startWaveOSC('q'+str(i+1), '/'+str(i+1)+'/q', waves[1])

def chordWalk():
    chordTree.execute(randWalk())
    sendChord(chordTree.currentNode.value)
                
def fun2():
    sendChord([3,7,11])
triggerFunctions["a"] = wavesWalk
triggerFunctions["b"] = chordWalk

timeSteps = [2.0, 2.0, 4.0, 4.0, 8.0] 
random.shuffle(timeSteps)
wavesRhythm = [sum(timeSteps[:i+1]) for i in range(len(timeSteps))]
random.shuffle(timeSteps)
chordsRhythm = [sum(timeSteps[:i+1]) for i in range(len(timeSteps))]


wavesPat = pydalModule.FlatGenericPattern([[t, "a"] for t in wavesRhythm], "funcTrigger", lambda a:a)
wavesPat.frac = sum(timeSteps)
chordsPat = pydalModule.FlatGenericPattern([[t, "b"] for t in chordsRhythm], "funcTrigger", lambda a:a)
chordsPat.frac = sum(timeSteps)
ch1.play(wavesPat, metaInfo=[port])
ch2.play(chordsPat, metaInfo=[port])

In [5]:
ch1.stop()
ch2.stop()

In [2]:
import random
timeSteps = [2.0, 2.0, 4.0, 4.0, 8.0] 
random.shuffle(timeSteps)
wavesRhythm = [sum(timeSteps[:i]) for i in range(len(timeSteps))]
print wavesRhythm, timeSteps

[0, 8.0, 10.0, 12.0, 16.0] [8.0, 2.0, 2.0, 4.0, 4.0]


In [ ]:
wavesWalk()
chordWalk()

In [ ]:
def randWalk():
    return random.choice(["_"]*3 + ["\/:0", "\/:1", ">", "<", "^", "^"])

randWalk()
ceil(0.6)

In [ ]:
def fun1():
    sendChord([0,3,7])
def fun2():
    sendChord([3,7,11])
triggerFunctions["a"] = fun1
triggerFunctions["b"] = fun2
pat = pydalModule.FlatGenericPattern([[0.0, "a"], [5.0, "b"]], "funcTrigger", lambda a:a)
pat.frac = 10.0
ch1.play(pat, metaInfo=[port])
ch1.stop()

In [ ]:
bf = (Sin(0.01)+1)*5

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*120 + 640)/2
q1 = 1 + (Sin(0.1)+1)*30
wp.startWaveOSC('f1', '/1/freq', f1, metaInfo="warp")
wp.startWaveOSC('q1', '/1/q', q1)

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*120 + 640)*2
q2 = Sin(0.1)*1 + 2
wp.startWaveOSC('f2', '/2/freq', f2, metaInfo="warp")
wp.startWaveOSC('q2', '/2/q', q2)

f3 = (Sin(phase=Sin(2  * bf))*120 + 640)/3
q3 = Sin(0.1)*3 + 60*Saw(-0.07, phase=Sin(0.06))
wp.startWaveOSC('f3', '/3/freq', f3, metaInfo="warp")
wp.startWaveOSC('q3', '/3/q', q3)

In [ ]:
bf = (Sin(0.01)+1)*5

q = 10 + (Sin(freq=Sin(0.08)+2)+1.)/2. * 30

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*120 + 640)/2 
q1 = 4 + (Sin(0.2)+1.)/2. * 10
wp.startWaveOSC('f1', '/1/freq', mtof(67))
wp.startWaveOSC('q1', '/1/q', q1)

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*120 + 640)*2
q2 = 4 + (Sin(0.2, 0.7)+1.)/2. * 10
wp.startWaveOSC('f2', '/2/freq', mtof(60))
wp.startWaveOSC('q2', '/2/q', q2)

f3 = (Sin(phase=Sin(2  * bf))*120 + 640)/3
q3 = 4 + (Sin(0.2, 1.4)+1.)/2. * 10
wp.startWaveOSC('f3', '/3/freq', mtof(63))
wp.startWaveOSC('q3', '/3/q', q3)

In [6]:
#different wave settings for variation tree will simply be different combinations of the components found here

bf = (Sin(0.01)+1)*1

q = 10 + (Sin(freq=Sin(0.08)+2)+1.)/2. * 30

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*12 + 64) 
m1 =Sin(0.2)*(Sqr() + 1.)*12.
q1 = 0.1 + (Sin(0.1)+1.)/2. * 2

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*8 + 40)
m2 = Saw(0.1)*-12
q2 = 1 + (Sin(0.1, 0.7)+1.)/2. * 10

f3 = (Sin(phase=Sin(2  * bf))*6 + 50)
q3 = 1 + (Sin(0.1, 1.4)+1.)/2. * 10



wp.startWaveOSC('f1', '/1/freq', f1+m1, metaInfo="warp")
wp.startWaveOSC('q1', '/1/q', q)


wp.startWaveOSC('f2', '/2/freq', f2+m1, metaInfo="warp")
wp.startWaveOSC('q2', '/2/q', q2)

wp.startWaveOSC('f3', '/3/freq', Sin(0.1)*5. + 60, metaInfo="warp")
wp.startWaveOSC('q3', '/3/q', q)

In [ ]:
print Sin(0.1)*3 + 60*Saw(-0.07, phase=Sin(0.06))

In [ ]:
wp.stopWave('f1')
wp.stopWave('q1')
wp.stopWave('f2')
wp.stopWave('q2')
wp.stopWave('f3')
wp.stopWave('q3')

In [ ]:
wp.plotWave(f2+m1, 100, 200)

In [ ]:
p1 = Sin(phase=Tri(-1)) * 500. + Cos(Tri(0.1)*10) * 100. 
p1 = Cos(phase=Tri()) * 500. + Cos(Tri(0.1)*10) * 100.
wp.startWaveOSC('p1', '/p1', p1)
wp.startWaveOSC('p2', '/p2', p1)

In [ ]:
wp.stopWave('p1')
wp.stopWave('p2')

In [ ]:
wp.plotWave(f2+m1, 100, 200)